In [6]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense


In [7]:

# Load RAF-DB dataset
def load_raf_db(data_dir, target_size=(70, 70)):
    """
    Load images and labels from the RAF-DB dataset.
    Assumes folder names are numeric (1 to 7) and adjusts labels to start from 0.
    """
    images, labels = [], []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for file in os.listdir(label_dir):
                img_path = os.path.join(label_dir, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:  # Check if image is read successfully
                    img = cv2.resize(img, target_size)
                    images.append(img)
                    labels.append(int(label) - 1)  # Adjust labels to start from 0
    images = np.array(images).reshape(-1, target_size[0], target_size[1], 1) / 255.0
    labels = np.array(labels)
    return images, labels

# Paths to the dataset
train_dir = r"C:\Users\isar\Desktop\DATASET\train"
test_dir = r"C:\Users\isar\Desktop\DATASET\test"

# Load train and test data
X_train, y_train = load_raf_db(train_dir)
X_test, y_test = load_raf_db(test_dir)

# Check number of classes
num_classes = len(np.unique(y_train))
print(f"Number of classes: {num_classes}")
print(f"Unique labels in training data: {np.unique(y_train)}")
print(f"Unique labels in test data: {np.unique(y_test)}")

# Convert labels to categorical (one-hot encoding)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(70, 70, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(num_classes, activation='softmax')
])


Number of classes: 7
Unique labels in training data: [0 1 2 3 4 5 6]
Unique labels in test data: [0 1 2 3 4 5 6]


In [8]:

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 100
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_test, y_test),
    epochs=epochs,
    steps_per_epoch=len(X_train) // batch_size,
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 49s 122ms/step - accuracy: 0.3689 - loss: 1.6741 - val_accuracy: 0.4377 - val_loss: 1.4599
Epoch 2/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4375 - loss: 1.5798 - val_accuracy: 0.4397 - val_loss: 1.4580
Epoch 3/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 46s 121ms/step - accuracy: 0.4384 - loss: 1.5099 - val_accuracy: 0.5336 - val_loss: 1.2639
Epoch 4/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6250 - loss: 1.3214 - val_accuracy: 0.5411 - val_loss: 1.2501
Epoch 5/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 46s 121ms/step - accuracy: 0.5027 - loss: 1.3618 - val_accuracy: 0.6043 - val_loss: 1.1116
Epoch 6/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5938 - loss: 1.2345 - val_accuracy: 0.6098 - val_loss: 1.1014
Epoch 7/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 46s 121ms/step - accuracy: 0.5441 - loss: 1.2629 - val_accuracy: 0.6392 - val_loss: 0.9948
Epoch 8/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5312 - loss: 1.144